<a href="https://colab.research.google.com/github/SuzanaKstro/Passo-a-passo/blob/main/C%C3%B3pia_de_Challenger03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark
!pip install findspark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
df = spark.read.option("header", "true").option("inferSchema", "true").csv("/content/PNAD_COVID_092020.csv")


In [ ]:
def renomear_colunas(df):
    df = df.withColumnRenamed("V1013", "nr_mes") \
        .withColumnRenamed("B0011", "teve_febre") \
        .withColumnRenamed("B0012", "teve_tosse") \
        .withColumnRenamed("B0013", "teve_garganta") \
        .withColumnRenamed("B0014", "teve_dif_respirar") \
        .withColumnRenamed("B00111", "teve_perda_olfato_paladar") \
        .withColumnRenamed("B011", "medida_contato") \
        .withColumnRenamed("B005", "ficou_internado") \
        .withColumnRenamed("A002", "nr_idade") \
        .withColumnRenamed("A003", "nm_sexo") \
        .withColumnRenamed("A004", "nm_raca") \
        .withColumnRenamed("B002", "procurou_Unidade_saude") \
        .withColumnRenamed("B008", "fez_teste_covid") \
        .withColumnRenamed("V1023", "tipo_area") \
        .withColumnRenamed("F002A5", "itens_higiene") \
        .withColumnRenamed("D0051", "recebeu_auxilio_emergencial") \
        .withColumnRenamed("E001", "solicitou_emprestimo") \
        .withColumnRenamed("A005", "nm_escolaridade") \
        .withColumnRenamed("C007C", "ocupacao_princial") \
        .withColumnRenamed("C007", "esta_empregado") \
        .withColumnRenamed("UF", "id_uf") \
        .withColumnRenamed("B009B", "positivo_swab") \
        .withColumnRenamed("B009D", "positivo_furo") \
        .withColumnRenamed("B009F", "positivo_braco")
    return df

def remover_colunas(df):
    colunas_para_manter = [
        "teve_febre",
        "teve_tosse",
        "teve_garganta",
        "teve_dif_respirar",
        "teve_perda_olfato_paladar",
        "medida_contato",
        "ficou_internado",
        "nr_idade",
        "nm_sexo",
        "nm_raca",
        "procurou_Unidade_saude",
        "fez_teste_covid",
        "tipo_area",
        "itens_higiene",
        "recebeu_auxilio_emergencial",
        "solicitou_emprestimo",
        "nm_escolaridade",
        "ocupacao_princial",
        "esta_empregado",
        "id_uf",
        "positivo_swab",
        "positivo_furo",
        "positivo_braco"
    ]
    df_reduzido = df.select(*colunas_para_manter)
    return df_reduzido


df = renomear_colunas(df)


df_reduzido = remover_colunas(df)


df_reduzido.show()


+----------+----------+-------------+-----------------+-------------------------+--------------+---------------+--------+-------+-------+----------------------+---------------+---------+-------------+---------------------------+--------------------+---------------+-----------------+--------------+-----+-------------+-------------+--------------+
|teve_febre|teve_tosse|teve_garganta|teve_dif_respirar|teve_perda_olfato_paladar|medida_contato|ficou_internado|nr_idade|nm_sexo|nm_raca|procurou_Unidade_saude|fez_teste_covid|tipo_area|itens_higiene|recebeu_auxilio_emergencial|solicitou_emprestimo|nm_escolaridade|ocupacao_princial|esta_empregado|id_uf|positivo_swab|positivo_furo|positivo_braco|
+----------+----------+-------------+-----------------+-------------------------+--------------+---------------+--------+-------+-------+----------------------+---------------+---------+-------------+---------------------------+--------------------+---------------+-----------------+--------------+-----+

In [ ]:
spark = SparkSession.builder.appName("desafio").getOrCreate()

In [ ]:
df_set = spark.read \
     .option("header", "true") \
     .option("inferSchema", "true") \
     .csv("/content/PNAD_COVID_092020.csv")


In [ ]:

df_out = spark.read \
     .option("header", "true") \
     .option("inferSchema", "true") \
     .csv("/content/PNAD_COVID_102020.csv")

In [ ]:
df_nov = spark.read \
     .option("header", "true") \
     .option("inferSchema", "true") \
     .csv("/content/PNAD_COVID_112020.csv")

In [ ]:
df_set_renomeado = renomear_colunas(df_set)

In [ ]:
df_out_renomeado = renomear_colunas(df_out)

In [ ]:
df_nov_renomeado = renomear_colunas(df_nov)

In [ ]:
df_set_reduzido = remover_colunas(df_set_renomeado)

In [ ]:
df_out_reduzido = remover_colunas(df_out_renomeado)

In [ ]:
df_nov_reduzido = remover_colunas(df_nov_renomeado)

In [ ]:
df_unido = df_set_reduzido.union(df_out_reduzido).union(df_nov_reduzido)

In [ ]:
df_unido_positivos = df_unido.filter((F.col("positivo_swab") == 1) | (F.col("positivo_furo") == 1) | (F.col("positivo_braco") == 1))

In [ ]:
df_unido_positivos.count()

32922

In [ ]:
df_procura_ativa_1 = df_unido.filter(
    (F.col("teve_febre") == 1) &
    (F.col("teve_tosse") == 1) &
    (F.col("teve_garganta") == 1) &
    (F.col("teve_dif_respirar") == 1) &
    (F.col("teve_perda_olfato_paladar") == 1) &
    (F.col("procurou_Unidade_saude") == 1))

In [ ]:
df_procura_ativa_1.count()

672

In [ ]:
df_procura_ativa_2 = df_unido.filter(
    (F.col("teve_febre") == 1) &
    (F.col("procurou_Unidade_saude") == 1))

In [ ]:
df_procura_ativa_2.count()

4909

In [ ]:
df_procura_ativa_3 = df_unido.filter(
    (F.col("teve_tosse") == 1) &
    (F.col("procurou_Unidade_saude") == 1))

In [ ]:
df_procura_ativa_3.count()

5488

In [ ]:
df_procura_ativa_4 = df_unido.filter(
    (F.col("teve_garganta") == 1) &
    (F.col("procurou_Unidade_saude") == 1))

In [ ]:
df_procura_ativa_4.count()

4828

In [ ]:
df_procura_ativa_5 = df_unido.filter(
    (F.col("teve_dif_respirar") == 1) &
    (F.col("procurou_Unidade_saude") == 1))

In [ ]:
df_procura_ativa_5.count()

2574

In [ ]:
df_procura_ativa_6 = df_unido.filter(

    (F.col("teve_perda_olfato_paladar") == 1) &
    (F.col("procurou_Unidade_saude") == 1))

In [ ]:
df_procura_ativa_6.count()

In [ ]:
df_procura_ativa = df_unido.filter(
    (F.col("teve_febre") == 1) &
    (F.col("teve_tosse") == 1) &
    (F.col("teve_garganta") == 1) &
    (F.col("teve_dif_respirar") == 1) &
    (F.col("teve_perda_olfato_paladar") == 1) &
    (F.col("procurou_Unidade_saude") == 2))

In [ ]:
df_procura_ativa.count()

In [ ]:
from pyspark.sql import functions as F

In [ ]:
df_positivos_por_escolaridade = df_unido_positivos.groupBy("nm_escolaridade").count()
df_positivos_por_escolaridade.show()

In [ ]:
df_escolaridade = df_positivos_por_escolaridade.withColumn(
    "nm_escolaridade",
    F.when(F.col("nm_escolaridade") == 1, "Sem instrução")
    .when(F.col("nm_escolaridade") == 2, "Fundamental incompleto")
    .when(F.col("nm_escolaridade") == 3, "Fundamental completo")
    .when(F.col("nm_escolaridade") == 4, "Médio incompleto")
    .when(F.col("nm_escolaridade") == 5, "Médio completo")
    .when(F.col("nm_escolaridade") == 6, "Superior incompleto")
    .when(F.col("nm_escolaridade") == 7, "Superior completo")
    .when(F.col("nm_escolaridade") == 8, "Pós-graduação")
    .otherwise("Desconhecido"))

In [ ]:
df_escolaridade.show()

In [ ]:
from pyspark.sql import functions as F


total_entrevistados_por_escolaridade = df_unido.groupBy("nm_escolaridade").count()


total_entrevistados_por_escolaridade = total_entrevistados_por_escolaridade.withColumn(
    "nm_escolaridade",
    F.when(F.col("nm_escolaridade") == 1, "Sem instrução")
    .when(F.col("nm_escolaridade") == 2, "Fundamental incompleto")
    .when(F.col("nm_escolaridade") == 3, "Fundamental completo")
    .when(F.col("nm_escolaridade") == 4, "Médio incompleto")
    .when(F.col("nm_escolaridade") == 5, "Médio completo")
    .when(F.col("nm_escolaridade") == 6, "Superior incompleto")
    .when(F.col("nm_escolaridade") == 7, "Superior completo")
    .when(F.col("nm_escolaridade") == 8, "Pós-graduação")
    .otherwise("Desconhecido"))


total_entrevistados_por_escolaridade.show()


In [ ]:
df_procura_ativa = df_unido.filter(F.col("procurou_Unidade_saude") == 2)


df_procura_escolaridade = df_procura_ativa.groupBy("nm_escolaridade").count()

df_procura_escolaridade.show()


In [ ]:
df_procura_escolaridade-procura_ativa = df_unido.filter(
    when(F.col("nm_escolaridade") == 1) / and(F.col("procurou_Unidade_saude") == 2))
    when(F.col("nm_escolaridade") == 2) / and(F.col("procurou_Unidade_saude") == 2))
    when(F.col("nm_escolaridade") == 3) / and(F.col("procurou_Unidade_saude") == 2))
    when(F.col("nm_escolaridade") == 4) / and(F.col("procurou_Unidade_saude") == 2))
    when(F.col("nm_escolaridade") == 5) / and(F.col("procurou_Unidade_saude") == 2))
    when(F.col("nm_escolaridade") == 6) /and(F.col("procurou_Unidade_saude") == 2))
    when(F.col("nm_escolaridade") == 7) /and(F.col("procurou_Unidade_saude") == 2))
    when(F.col("nm_escolaridade") == 8) /and(F.col("procurou_Unidade_saude") == 2))
    and(F.col("procurou_Unidade_saude") == 2))count

In [ ]:

df_emprestimo_ativa = df_unido.filter(F.col("solicitou_emprestimo") == 2)


df_procura_escolaridade_emprestimo = df_emprestimo_ativa.groupBy("nm_escolaridade").count()


df_procura_escolaridade_emprestimo.show()




In [ ]:
df_unido_positivos_emprestimo = df_unido.filter(
    ((F.col("positivo_swab") == 1) |
    (F.col("positivo_furo") == 1) |
    (F.col("positivo_braco") == 1)) &
    (F.col("solicitou_emprestimo") == 2)
)

df_unido_positivos_emprestimo.count()


In [ ]:
total_emprestimos = df.filter(F.col("solicitou_emprestimo") == 2).count()
print("Total de pessoas que solicitaram empréstimo:", total_emprestimos)


In [ ]:
from pyspark.sql import functions as F

# Filtrar os dados para obter apenas aqueles que testaram positivo
df_positivos = df_unido.filter(
    (F.col("positivo_swab") == 1) |
    (F.col("positivo_furo") == 1) |
    (F.col("positivo_braco") == 1)
)

# Agrupar por medida_contato e contar
df_medida_contato_positivos = df_positivos.groupBy("medida_contato").count()

# Adicionar as descrições das medidas de contato
df_medida_contato_positivos = df_medida_contato_positivos.withColumn(
    "descricao_medida_contato",
    F.when(F.col("medida_contato") == 1, "Não fez restrição")
    .when(F.col("medida_contato") == 2, "Reduziu o contato com as pessoas, mas continuou saindo de casa para trabalho")
    .when(F.col("medida_contato") == 3, "Ficou em casa e só saiu em caso de necessidade básica")
    .when(F.col("medida_contato") == 4, "Ficou rigorosamente em casa")
    .otherwise("Desconhecido")
)

# Ordenar os resultados em ordem crescente com base na medida de contato
df_medida_contato_positivos = df_medida_contato_positivos.orderBy(F.asc("medida_contato"))

# Mostrar os resultados
df_medida_contato_positivos.select("medida_contato", "descricao_medida_contato", "count").show(truncate=False)



In [ ]:
df_positivos = df_unido.filter(
    (F.col("positivo_swab") == 1) |
    (F.col("positivo_furo") == 1) |
    (F.col("positivo_braco") == 1)
)

df_itens_higiene_positivos = df_positivos.groupBy("itens_higiene").count()

df_itens_higiene_positivos.show(truncate=False)


In [ ]:
from pyspark.sql import functions as F

# Filtrar os dados para obter apenas aqueles que testaram positivo
df_positivos = df_unido.filter(
    (F.col("positivo_swab") == 1) |
    (F.col("positivo_furo") == 1) |
    (F.col("positivo_braco") == 1)
)

# Filtrar os positivos que receberam auxílio emergencial
df_positivos_auxilio = df_positivos.filter(F.col("recebeu_auxilio_emergencial") == 1)

# Agrupar por escolaridade e contar
df_auxilio_escolaridade = df_positivos_auxilio.groupBy("nm_escolaridade").count()

# Adicionar as descrições das escolaridades
df_auxilio_escolaridade = df_auxilio_escolaridade.withColumn(
    "descricao_escolaridade",
    F.when(F.col("nm_escolaridade") == 1, "Sem instrução")
    .when(F.col("nm_escolaridade") == 2, "Fundamental incompleto")
    .when(F.col("nm_escolaridade") == 3, "Fundamental completo")
    .when(F.col("nm_escolaridade") == 4, "Médio incompleto")
    .when(F.col("nm_escolaridade") == 5, "Médio completo")
    .when(F.col("nm_escolaridade") == 6, "Superior incompleto")
    .when(F.col("nm_escolaridade") == 7, "Superior completo")
    .when(F.col("nm_escolaridade") == 8, "Pós-graduação")
    .otherwise("Desconhecido")
)

# Mostrar os resultados
df_auxilio_escolaridade.select("nm_escolaridade", "descricao_escolaridade", "count").show(truncate=False)
